In [1]:
import polars as pl

In [18]:
pitching = pl.read_csv("data/pitching.csv")
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [19]:
eras = (pitching
        .select(['yearID', 'R'])
        .filter((pl.col('yearID') >= 1900) & (pl.col('yearID') < 1939))
        .with_columns(era = pl.when(pl.col('yearID') < 1920)
                              .then(pl.lit('dead ball'))
                              .otherwise(pl.lit('live ball'))
                     )
       )
eras.head()

yearID,R,era
i64,i64,str
1905,22,"""dead ball"""
1909,14,"""dead ball"""
1911,15,"""dead ball"""
1923,5,"""live ball"""
1920,40,"""live ball"""


In [20]:
mean_runs = (eras
             .group_by(pl.col('era'))
             .agg(mean_runs = pl.col('R').mean())
            )
mean_runs

era,mean_runs
str,f64
"""live ball""",55.76718
"""dead ball""",49.707696


In [38]:
pitching.with_columns(
    dh_in_league=(
        pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973)).then(pl.lit("yes"))
          .when(pl.col("yearID") == 2020).then(pl.lit("yes"))
          .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022)).then(pl.lit("yes"))
          .otherwise(pl.lit("no"))
          .alias("dh_in_league")
    )
).head()


playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,dh_in_league
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1,"""no"""
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2,"""no"""
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1,"""yes"""
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4,"""yes"""
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2,"""yes"""


In [45]:
ERA = (pitching
 .filter(pl.col("yearID") >= 1946)
 .group_by(pl.col("yearID", "lgID"))
 .agg
    (total_runs = pl.col("R").sum(),
     total_outs = pl.col("IPouts").sum()
    )
 .with_columns(ERA = (pl.col("total_runs") / pl.col("total_outs")) * 27)
 .with_columns(
    dh_in_league=(
        pl.when((pl.col("lgID") == "AL") & (pl.col("yearID") >= 1973)).then(pl.lit("yes"))
          .when(pl.col("yearID") == 2020).then(pl.lit("yes"))
          .when((pl.col("lgID") == "NL") & (pl.col("yearID") >= 2022)).then(pl.lit("yes"))
          .otherwise(pl.lit("no"))
          .alias("dh_in_league")
    )
)
    .select(["yearID", "lgID", "dh_in_league", "ERA"])
)
ERA

yearID,lgID,dh_in_league,ERA
i64,str,str,f64
2018,"""AL""","""yes""",4.601659
1984,"""AL""","""yes""",4.449852
1965,"""AL""","""no""",3.954239
1997,"""AL""","""yes""",4.980262
1953,"""AL""","""no""",4.485217
…,…,…,…
1989,"""AL""","""yes""",4.340122
1966,"""NL""","""no""",4.096944
2009,"""NL""","""no""",4.529872


In [ ]:
ERA.write_csv("DH.csv")